In [7]:
from processing import get_A_CH
from processing import get_P_CH

fits_file = r"D:\Data\EUV\193\2017\aia.lev1_5_euv_12s.2017-01-30T120006Z.193.image_lev1_5.fits"

In [8]:
A_CH = get_A_CH(fits_file)
print(A_CH)

(<Time object: scale='utc' format='isot' value=2017-01-30T12:00:04.846>, 0.4057118970264139)


In [9]:
print(A_CH[0], A_CH[1])

2017-01-30T12:00:04.846 0.4057118970264139


In [10]:
P_CH_30 = get_P_CH(fits_file)
P_CH_90 = get_P_CH(fits_file, lon=7.5, lat=90)
print(P_CH_30)
print(P_CH_90)

(<Time object: scale='utc' format='isot' value=2017-01-30T12:00:04.846>, 18033.83353508105)
(<Time object: scale='utc' format='isot' value=2017-01-30T12:00:04.846>, 27649.09364136569)


In [11]:
fits_file = r"D:\Data\EUV\193\2017\aia.lev1_5_euv_12s.2017-01-28T120006Z.193.image_lev1_5.fits"
A_CH = get_A_CH(fits_file)
P_CH_30 = get_P_CH(fits_file)
P_CH_90 = get_P_CH(fits_file, lon=7.5, lat=90)
print(A_CH)
print(P_CH_30)
print(P_CH_90)

(<Time object: scale='utc' format='isot' value=2017-01-28T12:00:04.843>, 0.027339411222224892)
(<Time object: scale='utc' format='isot' value=2017-01-28T12:00:04.843>, 9995.355801037378)
(<Time object: scale='utc' format='isot' value=2017-01-28T12:00:04.843>, 12736.156412926932)


In [1]:
import numpy as np
import pandas as pd

import argparse
from pathlib import Path
from tqdm import tqdm

from sunpy.time import parse_time
from datetime import datetime, timedelta
from multiprocessing import Pool

from processing import get_A_CH
from processing import get_P_CH

import warnings

In [57]:
channel = '193'
start = "2012-01-02"
end = "2013-02-21"
cadence = 12
base_dir = "D:\\Data\\EUV"
save_dir = "D:\\Data\\EUV"
cores = 4

In [58]:
base_dir = Path(base_dir)
save_dir = Path(save_dir)
start_dt = parse_time(start).to_datetime()
end_dt = parse_time(end).to_datetime()
channels = [chan.strip() for chan in channel.split(',')]   # e.g., [193,211]
years = range(start_dt.year, end_dt.year + 1)

In [119]:
def get_parameter(file):
    if file.exists():
        try:
            #a_ch = get_A_CH(file)
            p_ch30 = get_P_CH(file, lon=10, lat=30)
            p_ch90 = get_P_CH(file, lon=10, lat=90)
        except Exception:
            _, p_ch30, p_ch90 = np.nan, np.nan, np.nan
    else:
        _, p_ch30, p_ch90 = np.nan, np.nan, np.nan
    return np.nan, p_ch30, p_ch90

get_parameter(Path("D:/Data/EUV/193/2012/aia.lev1_5_euv_12s.2012-01-01T000009Z.193.image_lev1_5.fits"))

(nan,
 (<Time object: scale='utc' format='isot' value=2012-01-01T00:00:07.838>,
  4206.427192764431),
 (<Time object: scale='utc' format='isot' value=2012-01-01T00:00:07.838>,
  9053.077741189918))

In [139]:
for chan in channels:
    for year in years:
        source_dir = base_dir / str(chan) / str(year)
        destination_dir = save_dir
        destination_dir.mkdir(parents=True, exist_ok=True)
            
        save_file = destination_dir / "CH_Indics.csv"
         # --- resume logic: read existing times ---
        if save_file.exists():
            df = pd.read_csv(save_file)
            df.columns = df.columns.str.strip()
            if 'datetime' not in df.columns:
                alt = [c for c in df.columns if c.strip().lower() == 'datetime']
                if alt:
                    df.rename(columns={alt[0]: 'datetime'}, inplace=True)
                else:
                    raise KeyError(
                        f"there are no 'datetime' column. "
                        f"column in the files: {df.columns.tolist()}"
                    )
            df['datetime'] = pd.to_datetime(
                df['datetime'],
                format="%Y-%m-%d_%H",
                errors='coerce'
            )
            processed = set(df['datetime'].dt.to_pydatetime())
        else:
            save_file.write_text("datetime,A_CH,P_CH30,P_CH90\n")
            processed = set()

        year_start = max(start_dt, datetime(year, 1, 1, 0, 0))
        year_end   = min(end_dt,   datetime(year, 12, 31, 23, 59, 59))
        current = year_start
        dt_list = []
        while current <= year_end:
            dt_list.append(current)
            current += timedelta(hours=cadence)
            
        to_do = [dt for dt in dt_list if dt not in processed]
        
        def process_dt(dt):
            prefix = dt.strftime('%Y-%m-%dT%H')
            fname = f"aia.lev1_5_euv_12s.{prefix}*Z.{chan}.image_lev1_5.fits"
            matches = list(source_dir.glob(fname))
            if matches:
                fpath = matches[0]
            else:
                # dummy path to trigger nan in get_parameter
                fpath = source_dir / 'file_not_found.fits'
            a_ch, p_ch30, p_ch90 = get_parameter(fpath)
            return dt, a_ch, p_ch30, p_ch90
        
        

C:\Users\AHN\AppData\Local\Temp\ipykernel_19612\58805117.py:26: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  processed = set(df['datetime'].dt.to_pydatetime())
C:\Users\AHN\AppData\Local\Temp\ipykernel_19612\58805117.py:26: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  processed = set(df['datetime'].dt.to_pydatetime())


In [136]:

prefix = to_do[100].strftime('%Y-%m-%dT%H')
fname = f"aia.lev1_5_euv_12s.{prefix}*Z.{chan}.image_lev1_5.fits"
matches = list(source_dir.glob(fname))
print(to_do[100])

2013-02-20 00:00:00


In [82]:
print(files)

[WindowsPath('D:/Data/EUV/193/2013/aia.lev1_5_euv_12s.2013-01-01T000008Z.193.image_lev1_5.fits'), WindowsPath('D:/Data/EUV/193/2013/aia.lev1_5_euv_12s.2013-01-01T120008Z.193.image_lev1_5.fits'), WindowsPath('D:/Data/EUV/193/2013/aia.lev1_5_euv_12s.2013-01-02T000008Z.193.image_lev1_5.fits'), WindowsPath('D:/Data/EUV/193/2013/aia.lev1_5_euv_12s.2013-01-02T120008Z.193.image_lev1_5.fits'), WindowsPath('D:/Data/EUV/193/2013/aia.lev1_5_euv_12s.2013-01-03T000008Z.193.image_lev1_5.fits'), WindowsPath('D:/Data/EUV/193/2013/aia.lev1_5_euv_12s.2013-01-03T120008Z.193.image_lev1_5.fits'), WindowsPath('D:/Data/EUV/193/2013/aia.lev1_5_euv_12s.2013-01-04T000008Z.193.image_lev1_5.fits'), WindowsPath('D:/Data/EUV/193/2013/aia.lev1_5_euv_12s.2013-01-04T120008Z.193.image_lev1_5.fits'), WindowsPath('D:/Data/EUV/193/2013/aia.lev1_5_euv_12s.2013-01-05T000008Z.193.image_lev1_5.fits'), WindowsPath('D:/Data/EUV/193/2013/aia.lev1_5_euv_12s.2013-01-05T120008Z.193.image_lev1_5.fits'), WindowsPath('D:/Data/EUV/193/

In [16]:
fits_file = r"D:\Data\EUV\193\2012\aia.lev1_5_euv_12s.2012-01-12T120009Z.193.image_lev1_5.fits"
A_CH = get_A_CH(fits_file)
P_CH_30 = get_P_CH(fits_file)
P_CH_90 = get_P_CH(fits_file, lon=10, lat=90)
print(A_CH)
print(P_CH_30)
print(P_CH_90)

(<Time object: scale='utc' format='isot' value=2012-01-12T12:00:07.838>, 0.16891236093852252)
(<Time object: scale='utc' format='isot' value=2012-01-12T12:00:07.838>, 10435.969120715532)
(<Time object: scale='utc' format='isot' value=2012-01-12T12:00:07.838>, 15108.416876400413)


In [103]:
get_parameter(files[0])

((<Time object: scale='utc' format='isot' value=2013-01-01T00:00:06.838>, 0.0),
 (<Time object: scale='utc' format='isot' value=2013-01-01T00:00:06.838>,
  4807.738851241318),
 (<Time object: scale='utc' format='isot' value=2013-01-01T00:00:06.838>,
  9324.642118566711))